In [2]:
import pypsa
import matplotlib.pyplot as plt
import numpy as np

In [3]:
n_bus = 3
network = pypsa.Network()

# add three buses in the network
for i in range(n_bus):
    network.add("Bus", f"Bus {i}", v_nom = 20.0)

# print bus details
print(network.buses, "\n\n")

# add lines between the buses
for i in range(n_bus):
    network.add("Line", f"Line {i}{(i+1)%3}", bus0 = f"Bus {i}", bus1 = f"Bus {(i+1)%n_bus}", x = 0.1, r = 0.01)

attribute  v_nom type    x    y carrier unit  v_mag_pu_set  v_mag_pu_min  \
Bus                                                                        
Bus 0       20.0       0.0  0.0      AC                1.0           0.0   
Bus 1       20.0       0.0  0.0      AC                1.0           0.0   
Bus 2       20.0       0.0  0.0      AC                1.0           0.0   

attribute  v_mag_pu_max control generator sub_network  
Bus                                                    
Bus 0               inf      PQ                        
Bus 1               inf      PQ                        
Bus 2               inf      PQ                         




In [4]:
print(network.lines, "\n\n")

attribute   bus0   bus1 type    x     r    g    b  s_nom  s_nom_mod  \
Line                                                                  
Line 01    Bus 0  Bus 1       0.1  0.01  0.0  0.0    0.0        0.0   
Line 12    Bus 1  Bus 2       0.1  0.01  0.0  0.0    0.0        0.0   
Line 20    Bus 2  Bus 0       0.1  0.01  0.0  0.0    0.0        0.0   

attribute  s_nom_extendable  ...  v_ang_min  v_ang_max  sub_network  x_pu  \
Line                         ...                                            
Line 01               False  ...       -inf        inf                0.0   
Line 12               False  ...       -inf        inf                0.0   
Line 20               False  ...       -inf        inf                0.0   

attribute  r_pu  g_pu  b_pu x_pu_eff  r_pu_eff  s_nom_opt  
Line                                                       
Line 01     0.0   0.0   0.0      0.0       0.0        0.0  
Line 12     0.0   0.0   0.0      0.0       0.0        0.0  
Line 20     0.0   

In [5]:
# add a generator at bus 0
network.add("Generator", "Generator 1", bus = "Bus 0", p_set = 100, control = "PQ")
print(network.generators.p_set, "\n\n")

Generator
Generator 1    100.0
Name: p_set, dtype: float64 




In [6]:
network.add("Load", "Load 1", bus = "Bus 1", p_set = 100)
print(network.loads, "\n\n")
print(network.loads.p_set, "\n\n")

attribute    bus carrier type  p_set  q_set  sign
Load                                             
Load 1     Bus 1               100.0    0.0  -1.0 


Load
Load 1    100.0
Name: p_set, dtype: float64 




In [8]:
# perform the newton raphson load flow
network.pf()


INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.024424 seconds


{'n_iter': SubNetwork  0
 snapshot     
 now         3,
 'error': SubNetwork             0
 snapshot                
 now         2.004701e-12,
 'converged': SubNetwork     0
 snapshot        
 now         True}

In [9]:

# what is the active power flow in the lines?
print("active power flow through the lines")
print(network.lines_t.p0, "\n\n")

# ..and what are the voltage angle on the buses?
print("voltage angle on the buses")
print(network.buses_t.v_ang*180/np.pi, "\n\n")

# print the magnitudes of bus voltages in per unit(pu)
print("magnitudes of bus voltages in per unit(pu)")
print(network.buses_t.v_mag_pu, "\n\n")

active power flow through the lines
            Line 01    Line 12    Line 20
snapshot                                 
now       66.778181 -33.333333 -33.361212 


voltage angle on the buses
Bus       Bus 0     Bus 1     Bus 2
snapshot                           
now         0.0 -0.956705 -0.477919 


magnitudes of bus voltages in per unit(pu)
Bus       Bus 0     Bus 1     Bus 2
snapshot                           
now         1.0  0.998191  0.999061 


